# Exploratory Data Analysis II4091
## NIM: 18221095 
## Nama: Nicolas Owen Halim

######

### Import Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt

### Load Dataset

In [6]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.precision',10)

In [7]:
cs_df = pd.read_csv("dataset/train.csv")

/var/folders/2y/62bvs2vn73d5dy_1dvy2tb_w0000gn/T/ipykernel_7165/1143574855.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  cs_df = pd.read_csv("dataset/train.csv")


### One Hot Encoding Loan Type

In [9]:
loan_type_set = set()
for loan_types in cs_df.Type_of_Loan.unique():
    for loan_type in str(loan_types).split(","):
        loan_type = loan_type.removeprefix(" and").strip()
    loan_type_set.add(loan_type)

loan_type_list = list(loan_type_set)

for loan_type_col in loan_type_list:
    cs_df[loan_type_col] = 0

# Populate the new columns
def assign_loan_type_flags(row):
    loan_types_str = row['Type_of_Loan']
    current_loan_types = []
    # Split and clean the string for the current row
    for lt in str(loan_types_str).split(","):
        loan_type = lt.removeprefix(" and").strip()
        current_loan_types.append(loan_type)

    # Set the flag to 1 for relevant loan types in the current row
    for loan_type in current_loan_types:
        if loan_type in loan_type_list: # Check if it's a recognized type
            row[loan_type] = 1
    return row

cs_df = cs_df.apply(assign_loan_type_flags, axis=1)
cs_df["No Data"] = cs_df["nan"]
cs_df = cs_df.drop(["Type_of_Loan","nan"], axis=1)

In [10]:
cs_df.head(3)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,3,7,11.27,4.0,_,809.98,26.8226196237,22 Years and 1 Months,No,49.5749492149,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good,0,0,0,1,1,0,0,1,1,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,-1,NaN,11.27,4.0,Good,809.98,31.9449600554,NaN,No,49.5749492149,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good,0,0,0,1,1,0,0,1,1,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,3,7,_,4.0,Good,809.98,28.6093520221,22 Years and 3 Months,No,49.5749492149,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good,0,0,0,1,1,0,0,1,1,0


### Fix Num_of_Loan Column

Kolom Num_of_Loan merupakan hasil penjumlahan tipe pinjaman yang dilakukan dalam satu permohonan pinjaman. Pada kolom ini terdapat beberapa nilai yang salah seperti 316,737, 238, 132, dan 995. Hal ini tidak sesuai dengan kondisi dataset yang hanya terdapat 9 loan type. Oleh karena itu akan diubah jumlah num of loan yang tidak sesuai.

In [13]:
cs_df["Num_of_Loan"].value_counts().sort_values(ascending=True).head(5)

Num_of_Loan
316    1
737    1
238    1
132    1
995    1
Name: count, dtype: int64

In [14]:
cs_df[cs_df["No Data"] == 1]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
32,0x1632,CUS_0x1cdb,January,Deepaa,21,615-06-7821,Developer,35547.71_,2853.3091666667,7,5,5,0,5,NaN,2.58,4.0,Standard,943.86,39.7977642320,30 Years and 8 Months,Yes,0.0,276.72539431736266,!@9#%8,288.60552234930395,Standard,0,0,0,0,0,0,0,0,0,1
33,0x1633,CUS_0x1cdb,February,Deepaa,21,615-06-7821,Developer,35547.71,NaN,7,5,5,0,9,NaN,2.58,4.0,Standard,943.86,27.0203602539,30 Years and 9 Months,NM,0.0,74.44364104999623,High_spent_Medium_value_payments,460.88727561667037,Standard,0,0,0,0,0,0,0,0,0,1
34,0x1634,CUS_0x1cdb,March,Deepaa,21,615-06-7821,Developer,35547.71,2853.3091666667,7,5,5,-100,5,12,2.58,4.0,Standard,943.86,23.4623030364,30 Years and 10 Months,Yes,0.0,173.13865100158367,Low_spent_Medium_value_payments,392.1922656650829,Standard,0,0,0,0,0,0,0,0,0,1
35,0x1635,CUS_0x1cdb,April,Deepaa,21,615-06-7821,Developer,35547.71,2853.3091666667,7,5,5,0,1,15,2.58,4.0,_,943.86,28.9249537781,30 Years and 11 Months,Yes,0.0,96.78548508587444,High_spent_Medium_value_payments,438.5454315807922,Standard,0,0,0,0,0,0,0,0,0,1
36,0x1636,CUS_0x1cdb,May,Deepaa,21,615-06-7821,Developer,35547.71,2853.3091666667,7,5,5,0,9,17,2.58,4.0,_,943.86,41.7761871506,31 Years and 0 Months,Yes,0.0,62.72327834435009,High_spent_Small_value_payments,482.6076383223166,Standard,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99939,0x25f95,CUS_0xad4f,April,Sabina Zawadzkig,47,226-45-0652,Developer,22620.79,1722.0658333333,7,3,9,0,27,19_,5.31,2.0,_,642.46,26.7308625463,29 Years and 6 Months,NM,0.0,205.75607405146,Low_spent_Small_value_payments,256.4505092819,Poor,0,0,0,0,0,0,0,0,0,1
99940,0x25f96,CUS_0xad4f,May,Sabina Zawadzkig,47,226-45-0652,Developer,22620.79,1722.0658333333,7,3,9,0_,30,18,4.31,2.0,Standard,642.46,37.4507934806,29 Years and 7 Months,No,0.0,141.895743423042,Low_spent_Large_value_payments,300.3108399103,Standard,0,0,0,0,0,0,0,0,0,1
99941,0x25f97,CUS_0xad4f,June,Sabina Zawadzkig,47,226-45-0652,Developer,22620.79,1722.0658333333,7,3,9,0,27,18,5.31,2.0,Standard,642.46,27.6995042697,NaN,No,0.0,207.47345558788197,Low_spent_Small_value_payments,254.7331277455,Poor,0,0,0,0,0,0,0,0,0,1
99942,0x25f98,CUS_0xad4f,July,Sabina Zawadzkig,48_,#F%$D@*&8,Developer,22620.79,1722.0658333333,6,2,9,0,27,17,1.3099999999999996,2.0,Standard,642.46,24.7147786466,29 Years and 9 Months,No,0.0,16.53218878920387,High_spent_Large_value_payments,395.6743945441,Poor,0,0,0,0,0,0,0,0,0,1


In [15]:
loan_types=[
    "Debt Consolidation Loan",
    "Auto Loan","Payday Loan",
    "Mortgage Loan",
    "Credit-Builder Loan",
    "Student Loan",
    "Personal Loan",
    "Home Equity Loan"]
cs_df["Num_of_Loan"] = cs_df[loan_types].sum(axis=1)

### Fixing Age Column

Diketahui bahwa kolom age yang seharusnya memiliki tipe data integer menjadi tipe data object karena ada beberapa kesalahan value yang mengandung string "-" dan "_". Oleh karena itu, akan dihapus string tersebut agar dapat diubah menjadi bentuk integer

In [18]:
cs_df["Age"] = cs_df["Age"].apply(lambda x: x.rstrip("_").lstrip("-"))
print('Median:', cs_df["Age"].astype(int).median())

temp_df = cs_df["Age"].astype(int)
temp_df[temp_df > 34].value_counts().sort_index(ascending=True).loc[lambda x:x< 5]

Median: 34.0


Age
95      3
99      1
100     1
102     1
109     1
       ..
8674    1
8678    1
8682    1
8697    1
8698    1
Name: count, Length: 1684, dtype: int64

Bisa dilihat bahwa terdapat nilai anomali dimana ada beberapa usia yang hanya muncul beberapa kali dan bernilai sangat besar. Kita akan mencoba melihat beberapa sample dengan nilai anomali ini

In [20]:
cs_df[cs_df["Age"] == "95"]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
38248,0xf61e,CUS_0x7e94,January,Costas Pitasw,95,446-63-0451,Engineer,46162.62,3548.8850000000,10,5,16,2,43,19,6.45,6.0,Bad,1788.63,35.3864191638,15 Years and 5 Months,Yes,65.9568194085,288.64584382244783,Low_spent_Small_value_payments,290.28583676908613,Standard,0,0,0,0,0,1,0,1,0,0
65420,0x19552,CUS_0xdc8,May,NaN,95,362-50-3134,_______,28169.69,2171.4741666667,7,3,15,0,13,16,_,8.0,_,330.54,28.9010248153,29 Years and 1 Months,No,0.0000000000,146.1667164375651,Low_spent_Large_value_payments,340.9807002291016,Standard,0,0,0,0,0,0,0,0,0,1
78564,0x1e256,CUS_0x1dd3,May,Gerryg,95,936-83-6588,Manager,68484.44,NaN,5,7,5,3,12,20,12.52,8.0,Standard,105.41,28.4300667277,12 Years and 5 Months,NM,225.5356673408,220.00080637187816,High_spent_Small_value_payments,382.86719295400826,Standard,0,0,0,1,0,0,1,1,1,0


In [21]:
pd.concat([cs_df[cs_df["Customer_ID"] == "CUS_0x7e94"].head(3),cs_df[cs_df["Customer_ID"] == "CUS_0xdc8"].tail(5)])

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
38248,0xf61e,CUS_0x7e94,January,Costas Pitasw,95,446-63-0451,Engineer,46162.62,3548.8850000000,10,5,16,2,43,19,6.45,6.0,Bad,1788.63,35.3864191638,15 Years and 5 Months,Yes,65.9568194085,288.64584382244783,Low_spent_Small_value_payments,290.28583676908613,Standard,0,0,0,0,0,1,0,1,0,0
38249,0xf61f,CUS_0x7e94,February,Costas Pitasw,32,446-63-0451,Engineer,46162.62,3548.8850000000,10,5,16,2,45,19,2.45,6.0,Bad,1788.63,35.0936051785,15 Years and 6 Months,Yes,65.9568194085,51.315112747532005,High_spent_Medium_value_payments,487.61656784400185,Poor,0,0,0,0,0,1,0,1,0,0
38250,0xf620,CUS_0x7e94,March,Costas Pitasw,32,446-63-0451,_______,46162.62,3548.8850000000,10,5,16,2,38,17,6.45,7.0,_,1788.63,24.6549482900,15 Years and 7 Months,Yes,65.9568194085,163.21228343517177,High_spent_Small_value_payments,385.71939715636216,Poor,0,0,0,0,0,1,0,1,0,0
65419,0x19551,CUS_0xdc8,April,Jim Finkleu,51,362-50-3134,Doctor,28169.69,2171.4741666667,7,3,15,0,11,18,9.95,8.0,Standard,330.54,29.1497425740,29 Years and 0 Months,No,0.0000000000,179.62957447113703,Low_spent_Medium_value_payments,317.5178421955296,Standard,0,0,0,0,0,0,0,0,0,1
65420,0x19552,CUS_0xdc8,May,NaN,95,362-50-3134,_______,28169.69,2171.4741666667,7,3,15,0,13,16,_,8.0,_,330.54,28.9010248153,29 Years and 1 Months,No,0.0000000000,146.1667164375651,Low_spent_Large_value_payments,340.9807002291016,Standard,0,0,0,0,0,0,0,0,0,1
65421,0x19553,CUS_0xdc8,June,Jim Finkleu,51,362-50-3134,Doctor,28169.69,2171.4741666667,7,3,15,0,14,14,9.95,8.0,Standard,330.54,33.0078604692,29 Years and 2 Months,No,0.0000000000,47.56673538323561,High_spent_Large_value_payments,409.5806812834311,Standard,0,0,0,0,0,0,0,0,0,1
65422,0x19554,CUS_0xdc8,July,Jim Finkleu,51,362-50-3134,Doctor,28169.69,2171.4741666667,7,3,15,0,13,16,9.95,8.0,Standard,330.54,24.3813815357,29 Years and 3 Months,No,0.0000000000,118.35587437092056,Low_spent_Small_value_payments,388.7915422957462,Standard,0,0,0,0,0,0,0,0,0,1
65423,0x19555,CUS_0xdc8,August,Jim Finkleu,51,362-50-3134,Doctor,28169.69,2171.4741666667,7,3,15,0,14,15,10.95,8.0,Standard,330.54,28.6927913102,29 Years and 4 Months,No,0.0000000000,55.70537938599856,High_spent_Large_value_payments,401.4420372806681,Standard,0,0,0,0,0,0,0,0,0,1


Berdasarkan sample sebelumnya, bisa dilihat bahwa terdapat kesalahan penulisan usia yang berbeda dengan nilai modus usia yang dimiliki oleh Customer_ID X. Oleh karena itu, nilai anomali pada fitur X akan disubstitusikan dengan nilai pada row sebelum / sesudah sesuai dengan posisi nilai yang salah

In [23]:
age_anomalies = cs_df["Age"].astype(int) >= 95

for i in cs_df[age_anomalies].index:
    try:
        x = 1
        y = 1
        while ((cs_df.iloc[i]["Customer_ID"] == cs_df.iloc[i-x]["Customer_ID"]) and (int(cs_df.iloc[i]["Age"]) >= 95)):
            cs_df.at[i,"Age"] = cs_df.at[i-x,"Age"]
            x += 1
        while ((cs_df.iloc[i]["Customer_ID"] == cs_df.iloc[i+y]["Customer_ID"]) and (int(cs_df.iloc[i]["Age"]) >= 95)):
            cs_df.at[i,"Age"] = cs_df.at[i+y,"Age"]
            y += 1 
    except KeyError:
        pass

In [24]:
cs_df[cs_df["Customer_ID"] == "CUS_0x6794"]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
648,0x19ce,CUS_0x6794,January,Raymondl,41,204-14-2702,Teacher,103353.06,8527.755,3,3,6,1,4,9,3.11,2.0,Good,1374.56,32.6855216025,25 Years and 7 Months,No,64.0016856731,NaN,High_spent_Medium_value_payments,857.8262026487407,Good,0,0,1,0,0,0,0,0,0,0
649,0x19cf,CUS_0x6794,February,Raymondl,41,204-14-2702,_______,103353.06,8527.755,3,3,6,1,4,9,8.11,2.0,_,1374.56,40.5425517146,25 Years and 8 Months,No,64.0016856731,326.4827489349896,Low_spent_Medium_value_payments,742.2910653919014,Good,0,0,1,0,0,0,0,0,0,0
650,0x19d0,CUS_0x6794,March,Raymondl,41,204-14-2702,Teacher,103353.06,8527.755,3,3,6,1,4,7,3.11,2.0,Good,1374.56,33.9918543958,25 Years and 9 Months,No,64.0016856731,173.1808383657975,High_spent_Large_value_payments,855.5929759610934,Standard,0,0,1,0,0,0,0,0,0,0
651,0x19d1,CUS_0x6794,April,Raymondl,42,204-14-2702,Teacher,103353.06,8527.755,3,3,6,1,4,9,3.11,2.0,Good,1374.56,23.1708445819,25 Years and 10 Months,No,64.0016856731,1043.8736115710117,Low_spent_Small_value_payments,34.900202755879604,Good,0,0,1,0,0,0,0,0,0,0
652,0x19d2,CUS_0x6794,May,Raymondl,42,204-14-2702,Teacher,103353.06,8527.755,3,3,6,1,4,9,3.11,2.0,_,1374.56,38.7661597608,25 Years and 11 Months,No,64.0016856731,165.44837065048912,High_spent_Large_value_payments,863.325443676402,Good,0,0,1,0,0,0,0,0,0,0
653,0x19d3,CUS_0x6794,June,Raymondl,42,204-14-2702,Teacher,103353.06,8527.755,3,3,6,1,4,6,3.11,2.0,_,1374.56,28.6501269902,26 Years and 0 Months,No,64.0016856731,124.28948244015392,High_spent_Large_value_payments,904.4843318867372,Good,0,0,1,0,0,0,0,0,0,0
654,0x19d4,CUS_0x6794,July,Raymondl,42,204-14-2702,Teacher,103353.06,8527.755,3,3,6,1,2,9_,3.11,2.0,Good,1374.56,32.9988308960,26 Years and 1 Months,No,64.0016856731,1038.7009403582074,Low_spent_Small_value_payments,40.07287396868367,Good,0,0,1,0,0,0,0,0,0,0
655,0x19d5,CUS_0x6794,August,Raymondl,42,204-14-2702,Teacher,103353.06,8527.755,3,3,6,1,4,12,3.11,2.0,Good,1374.56,41.8370653324,26 Years and 2 Months,No,64.0016856731,67.10576784585814,High_spent_Large_value_payments,961.6680464810327,Good,0,0,1,0,0,0,0,0,0,0


In [25]:
cs_df["Age"] = cs_df["Age"].astype("int64")

### Fix Occupation

Pada tahap EDA, dapat dilihat bahwa terdapat sebuah nilai yang berbeda sendiri berupa "_______" pada kolom Occupation. Oleh karena itu, akan disubstitusikan value tersebut dengan nilai modus untuk masing-masing Customer_ID.

In [28]:
is_anomaly = cs_df["Occupation"] == "_______"

fix_values_map = cs_df[~is_anomaly].groupby('Customer_ID')['Occupation'].apply(lambda x: x.mode().iloc[0] if not x.empty else np.nan)

fix_series = cs_df['Customer_ID'].map(fix_values_map)

cs_df.loc[is_anomaly, 'Occupation'] = fix_series

In [29]:
cs_df[cs_df["Occupation"] == "_______"]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data


### Fix Num Bank Accounts

Untuk memperbaiki nilai pada kolom "Num_Bank_Accounts", perlu diperhatikan jumlah kemunculan masing-masing nilai. Dilakukan pemeriksaan untuk mencari nilai dengan frekuensi kemunculan diatas 5 kali. Dapat dilihat, nilai yang lebih kecil dari 10 memiliki frekuensi kemunculan yang jauh lebih besar dibandingkan dengan nilai lainnya. Melalui pemeriksaan ini, penulis menyimpulkan bahwa angka diatas 10 (kecuali 11) merupakan kesalahan input data dan perlu dilakukan perbaikan nilai. Selain itu, terdapat anomali dengan adanya nilai -1. Terakhir, perlu dilakukan analisis lebih lanjut untuk memeriksa nilai Num_Bank_Account 11 (nilai yang masih logis) karena kemunculannya yang sedikit.

In [32]:
value_counts = cs_df["Num_Bank_Accounts"].value_counts()

frequent_num_accounts = value_counts[value_counts > 5]
print(frequent_num_accounts.sort_index())

Num_Bank_Accounts
-1         21
 0       4328
 1       4490
 2       4304
 3      11950
 4      12186
 5      12118
 6      13001
 7      12823
 8      12765
 9       5443
 10      5247
 11         9
 803        7
Name: count, dtype: int64


In [33]:
eleven_bank_account = cs_df[cs_df["Num_Bank_Accounts"] == 11]["Customer_ID"].unique().tolist()

In [34]:
cs_df[cs_df["Customer_ID"].isin(eleven_bank_account)]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
76440,0x1d5e6,CUS_0x6cd1,January,Davidc,19,659-11-2212,Mechanic,81338.16_,6642.1800,10,9,33,5,60,15,23.26,12.0,Bad,4455.19,26.7225490487,13 Years and 10 Months,Yes,356.7665325320,137.63861680216834,High_spent_Large_value_payments,409.81285066587685,Poor,0,1,0,0,1,1,0,1,1,0
76441,0x1d5e7,CUS_0x6cd1,February,Davidc,19,659-11-2212,Mechanic,81338.16,6642.1800,10,9,33,5,60,15,19.26,12.0,Bad,4455.19,35.1516952145,13 Years and 11 Months,Yes,356.7665325320,143.5125510419868,High_spent_Large_value_payments,403.93891642605837,Poor,0,1,0,0,1,1,0,1,1,0
76442,0x1d5e8,CUS_0x6cd1,March,Davidc,19,659-11-2212,Mechanic,81338.16,6642.1800,10,9,33,5,60,12,19.26,12.0,Bad,4455.19,32.1505283813,14 Years and 0 Months,Yes,356.7665325320,73.17213387936616,High_spent_Medium_value_payments,484.27933358867904,Poor,0,1,0,0,1,1,0,1,1,0
76443,0x1d5e9,CUS_0x6cd1,April,NaN,19,659-11-2212,Mechanic,81338.16,6642.1800,10,1151,33,5,60,17,19.26,12.0,Bad,4455.19,40.3967073855,14 Years and 1 Months,Yes,356.7665325320,121.50057052145479,High_spent_Large_value_payments,425.9508969465903,Poor,0,1,0,0,1,1,0,1,1,0
76444,0x1d5ea,CUS_0x6cd1,May,NaN,19,659-11-2212,Mechanic,81338.16,NaN,11,10,33,5,65,15,_,NaN,Bad,4455.19,38.9204593018,14 Years and 2 Months,Yes,356.7665325320,166.49892146752126,High_spent_Large_value_payments,380.95254600052397,Standard,0,1,0,0,1,1,0,1,1,0
76445,0x1d5eb,CUS_0x6cd1,June,Davidc,19,659-11-2212,Mechanic,81338.16,6642.1800,11,10,33,5,60,NaN,22.26,12.0,Bad,4455.19,35.0394838273,14 Years and 3 Months,NM,356.7665325320,62.049075932509396,High_spent_Large_value_payments,485.4023915355358,Poor,0,1,0,0,1,1,0,1,1,0
76446,0x1d5ec,CUS_0x6cd1,July,NaN,19,659-11-2212,Mechanic,81338.16,6642.1800,11,10,33,5,60,15,_,12.0,Bad,4455.19,35.0654362899,14 Years and 4 Months,Yes,356.7665325320,359.8894705458568,High_spent_Small_value_payments,207.56199692218846,Standard,0,1,0,0,1,1,0,1,1,0
76447,0x1d5ed,CUS_0x6cd1,August,Davidc,20,659-11-2212,Mechanic,81338.16,6642.1800,11,10,33,5,60,15,19.26,12.0,_,4455.19,28.2611339423,14 Years and 5 Months,Yes,356.7665325320,441.52624142297316,Low_spent_Large_value_payments,135.925226045072,Poor,0,1,0,0,1,1,0,1,1,0
85952,0x20da2,CUS_0x81c5,January,Sakaria,26,817-22-9418,Teacher,55554.69,4649.5575,10,8,34,4,37,13_,21.76,11.0,Bad,4974.31,27.6312131155,0 Years and 10 Months,Yes,228.1167576017,36.05107515017643,High_spent_Large_value_payments,440.78791724814397,Standard,1,0,1,1,0,0,0,1,0,0
85953,0x20da3,CUS_0x81c5,February,NaN,26,817-22-9418,Teacher,55554.69,4649.5575,10,8,34,4,42,14,21.76,11.0,Bad,4974.31,32.9507863917,0 Years and 11 Months,Yes,228.1167576017,117.75709897567295,High_spent_Medium_value_payments,369.0818934226474,Standard,1,0,1,1,0,0,0,1,0,0


Berdasarkan hasil di atas, Customer_ID yang memiliki nilai 11 pada kolom "Num_Bank_Accounts" masih normal. Berdasarkan nilai kolom "Num_Bank_Accounts" pada baris lain yang bernilai 10, dapat disimpulkan bahwa pada baris lain akun bank customer bertambah 1. Oleh karena itu, kita akan memperbaiki baris lain yang memiliki nilai > 11 dan -1

In [36]:
is_anomaly = ((cs_df["Num_Bank_Accounts"] > 11) | (cs_df["Num_Bank_Accounts"] < 0))

fix_values_map = cs_df[~is_anomaly].groupby('Customer_ID')['Num_Bank_Accounts'].apply(lambda x: x.mode().iloc[0] if not x.empty else np.nan)

fix_series = cs_df['Customer_ID'].map(fix_values_map)

cs_df.loc[is_anomaly, 'Num_Bank_Accounts'] = fix_series

In [37]:
bank_account_anomaly = (cs_df["Num_Bank_Accounts"] > 11) | (cs_df["Num_Bank_Accounts"] < 0)
cs_df[bank_account_anomaly]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data


### Fix Payment Behaviour

Kolom payment behaviour memiliki beberapa data dengan nilai yang tidak memiliki arti, yaitu "!@9#%8". Oleh karena itu, kita akan mengganti nilai tersebut dengan metode yang sama dengan sebelumnya melalui modus Customer_ID.

In [40]:
cs_df["Payment_Behaviour"].unique().tolist()

['High_spent_Small_value_payments',
 'Low_spent_Large_value_payments',
 'Low_spent_Medium_value_payments',
 'Low_spent_Small_value_payments',
 'High_spent_Medium_value_payments',
 '!@9#%8',
 'High_spent_Large_value_payments']

In [41]:
is_anomaly = cs_df["Payment_Behaviour"] == "!@9#%8"

fix_values_map = cs_df[~is_anomaly].groupby('Customer_ID')['Payment_Behaviour'].apply(lambda x: x.mode().iloc[0] if not x.empty else np.nan)

fix_series = cs_df['Customer_ID'].map(fix_values_map)

cs_df.loc[is_anomaly, 'Payment_Behaviour'] = fix_series

# This method is much faster and automatically handles customers who may not have
# any "good" values to calculate a mode from (it will result in NaN, which you can handle later if needed).

In [42]:
cs_df[cs_df["Payment_Behaviour"] == "!@9#%8"]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data


### Fix Num Credit Card

Kolom Num_Credit_Card memiliki kondisi yang mirip dengan kolom Num_Bank_Account sehingga akan dilakukan pembersihan data dengan pendekatan yang sama

In [45]:
value_counts = cs_df["Num_Credit_Card"].value_counts()

frequent_num_accounts = value_counts[value_counts > 5]
print(frequent_num_accounts.sort_index())

Num_Credit_Card
0          13
1        2132
2        2149
3       13277
4       14030
5       18459
6       16559
7       16615
8        4956
9        4643
10       4860
11         36
92          6
106         6
159         6
183         6
218         6
331         6
849         8
852         7
958         6
1420        6
Name: count, dtype: int64


In [46]:
eleven_credit_card = cs_df[cs_df["Num_Credit_Card"] == 11]["Customer_ID"].unique().tolist()
cs_df[cs_df["Customer_ID"].isin(eleven_credit_card)]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
1000,0x1bde,CUS_0x3187,January,NaN,39,024-52-4439,Architect,9983.345,758.9454166667,6,10,15,3,22,16,10.26,11.0,Standard,1823.35,36.9882876894,9 Years and 2 Months,NM,29.2415186896,NaN,Low_spent_Small_value_payments,261.65548061715543,Poor,1,0,1,0,1,0,1,0,0,0
1001,0x1bdf,CUS_0x3187,February,Jan Schwartze,39,024-52-4439,Architect,9983.345,758.9454166667,6,10,15,3,22,16,10.26,11.0,_,1823.35,29.5094227425,9 Years and 3 Months,Yes,29.2415186896,18.18202076651747,High_spent_Medium_value_payments,278.4710022105077,Poor,1,0,1,0,1,0,1,0,0,0
1002,0x1be0,CUS_0x3187,March,Jan Schwartze,39,024-52-4439,Architect,9983.345,758.9454166667,7,11,15,3,22,16,12.26,11.0,Standard,1823.35,39.6377560114,9 Years and 4 Months,Yes,29.2415186896,47.6492134812721,Low_spent_Medium_value_payments,279.00380949575305,Standard,1,0,1,0,1,0,1,0,0,0
1003,0x1be1,CUS_0x3187,April,Jan Schwartze,39,#F%$D@*&8,Architect,9983.345,758.9454166667,7,11,15,3,22,16,17.259999999999998,11.0,Standard,1823.35,26.6119777793,9 Years and 5 Months,NM,29.2415186896,42.379896403360725,High_spent_Small_value_payments,264.2731265736644,Standard,1,0,1,0,1,0,1,0,0,0
1004,0x1be2,CUS_0x3187,May,Jan Schwartze,40,024-52-4439,Architect,9983.345,758.9454166667,7,11,15,3,22,NaN,14.26,11.0,_,1823.35,30.2748776859,NaN,Yes,29.2415186896,0.0,High_spent_Large_value_payments,277.7414585489011,Standard,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97403,0x250b9,CUS_0xebc,April,McMillant,33,022-23-5391,Media_Manager,14310.25,1064.5208333333,9,10,34,6,15,25,-0.46,6.0,Bad,2188.14,24.8822603722,12 Years and 2 Months,Yes,92.4140805435,36.097494497528686,High_spent_Medium_value_payments,227.9405082922892,Standard,0,0,1,1,1,1,0,1,1,0
97404,0x250ba,CUS_0xebc,May,McMillant,33,022-23-5391,Media_Manager,14310.25,1064.5208333333,9,10,34,6,11,26,1.54,6.0,Bad,2188.14,40.2225048006,NaN,Yes,92.4140805435,102.4225497846888,Low_spent_Small_value_payments,201.6154530051291,Standard,0,0,1,1,1,1,0,1,1,0
97405,0x250bb,CUS_0xebc,June,McMillant,33,022-23-5391,Media_Manager,14310.25,1064.5208333333,9,10,34,6,15,NaN,1.54,6.0,Bad,2188.14,38.5535748726,12 Years and 4 Months,Yes,92.4140805435,__10000__,High_spent_Large_value_payments,230.957837240058,Standard,0,0,1,1,1,1,0,1,1,0
97406,0x250bc,CUS_0xebc,July,McMillant,33,022-23-5391,Media_Manager,14310.25,1064.5208333333,9,10,34,6,15,28,1.54,6.0,Bad,2188.14,34.6341943870,12 Years and 5 Months,Yes,92.4140805435,74.99431805702129,Low_spent_Medium_value_payments,219.0436847327966,Poor,0,0,1,1,1,1,0,1,1,0


In [47]:
is_anomaly = cs_df["Num_Credit_Card"] > 11

fix_values_map = cs_df[~is_anomaly].groupby('Customer_ID')['Num_Credit_Card'].apply(lambda x: x.mode().iloc[0] if not x.empty else np.nan)

fix_series = cs_df['Customer_ID'].map(fix_values_map)

cs_df.loc[is_anomaly, 'Num_Credit_Card'] = fix_series

# This method is much faster and automatically handles customers who may not have
# any "good" values to calculate a mode from (it will result in NaN, which you can handle later if needed).

In [48]:
cs_df[cs_df["Num_Credit_Card"] > 11]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data


### Fix Credit History Age Column

Pada kolom "Credit_History_Age" terdapat beberapa row dengan missing values. Pada setiap Customer_ID, data "Credit_History_Age" bisa dilihat memiliki nilai yang berurutan .Oleh karena itu, nilai kosong akan disubstitusikan dengan nilai sebelum / sesudahnya sesuai dengan Customer_ID-nya. Terakhir, nilai pada kolom "Credit_History_Age" akan diubah ke dalam bentuk float sesuai dengan tahun ditambahkan dengan bulan

In [51]:
cs_df["Credit_History_Age"].unique().tolist()[0:15]

['22 Years and 1 Months',
 nan,
 '22 Years and 3 Months',
 '22 Years and 4 Months',
 '22 Years and 5 Months',
 '22 Years and 6 Months',
 '22 Years and 7 Months',
 '26 Years and 7 Months',
 '26 Years and 8 Months',
 '26 Years and 9 Months',
 '26 Years and 10 Months',
 '26 Years and 11 Months',
 '27 Years and 0 Months',
 '27 Years and 1 Months',
 '27 Years and 2 Months']

In [52]:
missing_credit_age = cs_df["Credit_History_Age"].isna()
for i in cs_df[missing_credit_age].index:
    try:
        x = 1
        y = 1
        while ((cs_df.iloc[i]["Customer_ID"] == cs_df.iloc[i-x]["Customer_ID"]) and (pd.isna(cs_df.iloc[i]["Credit_History_Age"]))):
            if (pd.notna(cs_df.iloc[i-x]["Credit_History_Age"])):
                history_age_list = cs_df.iloc[i-x]["Credit_History_Age"].split()
                history_age_list[3] = str(int(history_age_list[3]) + 1)
                cs_df.at[i,"Credit_History_Age"] = " ".join(history_age_list)
            else:
                x += 1
        while ((cs_df.iloc[i]["Customer_ID"] == cs_df.iloc[i+y]["Customer_ID"]) and (pd.isna(cs_df.iloc[i]["Credit_History_Age"]))):
            if (pd.notna(cs_df.iloc[i+y]["Credit_History_Age"])):
                history_age_list = cs_df.iloc[i+y]["Credit_History_Age"].split()
                history_age_list[3] = str(int(history_age_list[3]) + 1)
                cs_df.at[i,"Credit_History_Age"] = " ".join(history_age_list)
            else:
                y += 1 
    except KeyError:
        pass

In [53]:
cs_df[missing_credit_age].head(2)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,-1,NaN,11.27,4.0,Good,809.98,31.9449600554,22 Years and 2 Months,No,49.5749492149,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good,0,0,0,1,1,0,0,1,1,0
7,0x1609,CUS_0xd40,August,NaN,23,#F%$D@*&8,Scientist,19114.12,1824.8433333333,3,4,3,4,3,6,11.27,4.0,Good,809.98,23.9337948020,22 Years and 8 Months,No,49.5749492149,24.785216509052056,High_spent_Medium_value_payments,358.12416760938714,Standard,0,0,0,1,1,0,0,1,1,0


In [54]:
cs_df[cs_df["Customer_ID"] == "CUS_0xd40"]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,3,7,11.27,4.0,_,809.98,26.8226196237,22 Years and 1 Months,No,49.5749492149,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good,0,0,0,1,1,0,0,1,1,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,-1,NaN,11.27,4.0,Good,809.98,31.9449600554,22 Years and 2 Months,No,49.5749492149,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good,0,0,0,1,1,0,0,1,1,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,3,7,_,4.0,Good,809.98,28.6093520221,22 Years and 3 Months,No,49.5749492149,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good,0,0,0,1,1,0,0,1,1,0
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,5,4,6.27,4.0,Good,809.98,31.3778618696,22 Years and 4 Months,No,49.5749492149,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good,0,0,0,1,1,0,0,1,1,0
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,6,NaN,11.27,4.0,Good,809.98,24.7973469088,22 Years and 5 Months,No,49.5749492149,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good,0,0,0,1,1,0,0,1,1,0
5,0x1607,CUS_0xd40,June,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,8,4,9.27,4.0,Good,809.98,27.2622587105,22 Years and 6 Months,No,49.5749492149,62.430172331195294,High_spent_Medium_value_payments,340.4792117872438,Good,0,0,0,1,1,0,0,1,1,0
6,0x1608,CUS_0xd40,July,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,3,8_,11.27,4.0,Good,809.98,22.5375930318,22 Years and 7 Months,No,49.5749492149,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good,0,0,0,1,1,0,0,1,1,0
7,0x1609,CUS_0xd40,August,NaN,23,#F%$D@*&8,Scientist,19114.12,1824.8433333333,3,4,3,4,3,6,11.27,4.0,Good,809.98,23.9337948020,22 Years and 8 Months,No,49.5749492149,24.785216509052056,High_spent_Medium_value_payments,358.12416760938714,Standard,0,0,0,1,1,0,0,1,1,0


In [55]:
def convert_credit_age(data):
    temp_credit_age_list = data.split()
    year, month = int(temp_credit_age_list[0]), int(temp_credit_age_list[3])
    return year+month/12

cs_df["Credit_History_Age"] = cs_df["Credit_History_Age"].apply(convert_credit_age)
cs_df["Credit_History_Age"] = cs_df["Credit_History_Age"].astype(float)

### Fix Credit Mix Column

Pada kolom "Credit Mix", ada beberapa baris dengan nilai yang salah, yaitu "_". Hasil berikut menunjukkan bahwa setiap Customer_ID akan memiliki 1 nilai Credit Mix sehingga akan disubstitusikan dengan nilai credit mix sesuai dengan Customer_ID tersebut

In [58]:
print(cs_df["Credit_Mix"].unique().tolist())
print(cs_df[cs_df["Credit_Mix"] != "_"].groupby("Customer_ID")["Credit_Mix"].nunique().sort_values(ascending=False))

['_', 'Good', 'Standard', 'Bad']
Customer_ID
CUS_0x1000    1
CUS_0x8cfe    1
CUS_0x8cbe    1
CUS_0x8cc1    1
CUS_0x8cc5    1
             ..
CUS_0x4f3b    1
CUS_0x4f3e    1
CUS_0x4f41    1
CUS_0x4f43    1
CUS_0xffd     1
Name: Credit_Mix, Length: 12500, dtype: int64


In [59]:
is_anomaly = cs_df["Credit_Mix"] == "_"

fix_values_map = cs_df[~is_anomaly].groupby('Customer_ID')['Credit_Mix'].apply(lambda x: x.mode().iloc[0] if not x.empty else np.nan)

fix_series = cs_df['Customer_ID'].map(fix_values_map)

# 4. Use .loc to update the "Num_Credit_Inquiries" column ONLY for the anomalous rows
cs_df.loc[is_anomaly, 'Credit_Mix'] = fix_series

# This method is much faster and automatically handles customers who may not have
# any "good" values to calculate a mode from (it will result in NaN, which you can handle later if needed).

In [60]:
cs_df[cs_df["Credit_Mix"] == "_"]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data


### Fix Num Credit Inquiries Column

In [62]:
num_credit_inquiries_mean = cs_df["Num_Credit_Inquiries"].mean()
print("Num Credit Inquiries Mean:", num_credit_inquiries_mean)
print(sorted(cs_df[cs_df["Num_Credit_Inquiries"] < num_credit_inquiries_mean]["Num_Credit_Inquiries"].unique()))

Num Credit Inquiries Mean: 27.75425103279441
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 26.0]


In [63]:
cs_df[cs_df["Num_Credit_Inquiries"]  == 26]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
78527,0x1e21d,CUS_0x6534,August,Silvia Aloisit,14,#F%$D@*&8,Scientist,79697.48,6080.9659295738,7,7,26,5,47,25,22.03,26.0,Bad,2984.04,30.9035894203,5.5,Yes,841.8403869222,123.17714452627378,High_spent_Small_value_payments,301.8188723110387,Standard,1,1,0,1,0,1,1,0,1,0


In [64]:
# 1. Identify the anomalous rows
mean_inquiries = cs_df['Num_Credit_Inquiries'].mean()
is_anomaly = ((cs_df['Num_Credit_Inquiries'] > mean_inquiries) | cs_df["Num_Credit_Inquiries"].isna())

# 2. Create a mapping of each Customer_ID to their "good" mode value
# We only use the non-anomalous rows to calculate the fix values
fix_values_map = cs_df[~is_anomaly].groupby('Customer_ID')['Num_Credit_Inquiries'].apply(lambda x: x.mode().iloc[0] if not x.empty else np.nan)

# 3. Create a Series of fix values aligned with the original DataFrame
# For each row in cs_df, this looks up the fix value based on its Customer_ID
fix_series = cs_df['Customer_ID'].map(fix_values_map)

# 4. Use .loc to update the "Num_Credit_Inquiries" column ONLY for the anomalous rows
cs_df.loc[is_anomaly, 'Num_Credit_Inquiries'] = fix_series

# This method is much faster and automatically handles customers who may not have
# any "good" values to calculate a mode from (it will result in NaN, which you can handle later if needed).

In [65]:
cs_df[(cs_df["Num_Credit_Inquiries"] > 27) | cs_df["Num_Credit_Inquiries"].isna()]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data


### Fix Monthly Inhand Salary and Annual Income Column

Pertama, terdapat beberapa baris kosong pada kolom "Monthly_Inhand_Salary". Berdasarkan analisis yang dilakukan, masing-masing Customer_ID akan memiliki nilai "Monthly_Inhand_Salary" yang sama ataupun cukup mirip. Oleh karena itu, nilai kosong akan disubstitusikan dengan modus nilai Customer_ID tersebut.

Kedua, Kolom "Annual_Income" memiliki data yang cukup aneh, yaitu nilai annual_income yang sangat besar (melebihi 2x gaji tahunan) dibandingkan dengan hasil penjumlahan Monthly_Inhand_Salary. Selain itu, terdapat beberapa baris dengan nilai yang berbeda dengan modus berdasarkan Customer_ID tersebut. Oleh karena itu, kita akan mensubstitusikan nilai Annual_Income yang melebihi 25x nilai "Monthly_Inhand_Salary" (dengan asumsi masing-masing customer bekerja 12 bulan, mendapatkan bonus, dan ada pendapatan lain)

In [68]:
cs_df[cs_df["Monthly_Inhand_Salary"].notna()].groupby("Customer_ID")["Monthly_Inhand_Salary"].nunique().sort_values(ascending=False)

Customer_ID
CUS_0xbc9c    2
CUS_0xbba8    2
CUS_0xa503    2
CUS_0x4e5     2
CUS_0xa514    2
             ..
CUS_0x50d4    1
CUS_0x50d5    1
CUS_0x50dd    1
CUS_0x50df    1
CUS_0xffd     1
Name: Monthly_Inhand_Salary, Length: 12500, dtype: int64

In [69]:
cs_df[cs_df["Customer_ID"] == "CUS_0xbc9c"]["Monthly_Inhand_Salary"].unique()

array([5697.09666667, 5067.65788692])

In [70]:
monthly_salary_map = cs_df[cs_df["Monthly_Inhand_Salary"].notna()].groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(lambda x: x.mode().iloc[0] if not x.empty else np.nan)
cs_df["Monthly_Inhand_Salary"] = cs_df["Monthly_Inhand_Salary"].fillna(cs_df["Customer_ID"].map(monthly_salary_map))

In [71]:
cs_df["Annual_Income"] = cs_df["Annual_Income"].str.rstrip("_").astype(float)
is_anomaly = cs_df["Annual_Income"] > (25*cs_df["Monthly_Inhand_Salary"])


cs_df.loc[is_anomaly,"Annual_Income"] = round(cs_df["Monthly_Inhand_Salary"]*12,3) 

### Fix Interest_Rate Column

Berdasarkan analisis dataset, masing-masing customer hanya memiliki 1 nilai "Interest_Rate". Namun, terdapat beberapa kesalahan yang ditandai dengan  customer memiliki interest rate yang sangat jauh dari nilai modus-nya.

In [74]:
n_interest_rate_customer = cs_df.groupby("Customer_ID")["Interest_Rate"].nunique()
print("Customer with more than 1 interest rate:", n_interest_rate_customer[n_interest_rate_customer>1].count())
is_anomaly_example = n_interest_rate_customer.sort_values(ascending=False).head(5).index
for i in is_anomaly_example:
    print(f"{i}:")
    print(f"Interest Rates: {cs_df[cs_df['Customer_ID'] == i]['Interest_Rate'].unique().tolist()}")
    print(f"Mode Interest Rate: {cs_df[cs_df['Customer_ID'] == i]['Interest_Rate'].mode().iloc[0]}")
    print("")

Customer with more than 1 interest rate: 1911
CUS_0x44a4:
Interest Rates: [27, 5062, 2340, 5277]
Mode Interest Rate: 27

CUS_0x9962:
Interest Rates: [5580, 5, 1024, 1434]
Mode Interest Rate: 5

CUS_0xc6d1:
Interest Rates: [18, 146, 5546, 107]
Mode Interest Rate: 18

CUS_0x62b5:
Interest Rates: [6, 5425, 440, 1380]
Mode Interest Rate: 6

CUS_0x2bcb:
Interest Rates: [5, 4062, 2308, 3043]
Mode Interest Rate: 5



In [75]:
temp_df = cs_df.groupby("Customer_ID")["Interest_Rate"].agg(lambda x:x.mode().iloc[0]).rename("Mode_IR")
cs_df = cs_df.merge(temp_df,on="Customer_ID")
cs_df.loc[cs_df["Interest_Rate"] != cs_df["Mode_IR"],"Interest_Rate"] = cs_df["Mode_IR"]
cs_df = cs_df.drop("Mode_IR",axis=1)

In [76]:
cs_df.head(3)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,3,7,11.27,4.0,Good,809.98,26.8226196237,22.0833333333,No,49.5749492149,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good,0,0,0,1,1,0,0,1,1,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,-1,NaN,11.27,4.0,Good,809.98,31.9449600554,22.1666666667,No,49.5749492149,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good,0,0,0,1,1,0,0,1,1,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,3,7,_,4.0,Good,809.98,28.6093520221,22.2500000000,No,49.5749492149,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good,0,0,0,1,1,0,0,1,1,0


### Fix Amount_invested_monthly Column

Anomaly : \_\_10000\_\_

In [79]:
is_anomaly = (cs_df["Amount_invested_monthly"] == "__10000__")

In [80]:
print(f"Anomaly Count: {len(cs_df[is_anomaly])}")
cs_df[is_anomaly].head(3)

Anomaly Count: 4305


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
18,0x161c,CUS_0x2dbc,March,Langep,34,486-85-3974,Engineer,143162.64,12187.2200000000,1,5,8,1,8,7,11.1,3.0,Good,1303.01,26.5198153915,17.9166666667,No,246.9923194537,__10000__,High_spent_Small_value_payments,715.741367403555,Good,0,0,0,0,1,0,1,0,0,0
23,0x1621,CUS_0x2dbc,August,Langep,34,486-85-3974,Engineer,143162.64,12187.2200000000,1,5,8,1,8,6,7.1,3.0,Good,1303.01,38.3747529478,18.3333333333,No,246.9923194537,__10000__,High_spent_Small_value_payments,895.494583180492,Standard,0,0,0,0,1,0,1,0,0,0
28,0x162a,CUS_0xb891,May,Jasond,55,072-31-6145,Entrepreneur,30689.89,2612.4908333333,2,5,4,0,5,6,-1.01,4.0,Good,632.46,41.1543165470,17.5833333333,No,16.4154516598,__10000__,Low_spent_Large_value_payments,359.37491550776383,Standard,0,0,0,0,0,0,1,0,0,0


In [81]:
cs_df.loc[is_anomaly,"Amount_invested_monthly"] = np.nan
cs_df["Amount_invested_monthly"] = cs_df["Amount_invested_monthly"].astype("Float64")
cs_df["Amount_invested_monthly"] = cs_df.groupby("Customer_ID")["Amount_invested_monthly"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

In [82]:
cs_df[cs_df["Customer_ID"] == "CUS_0x2dbc"].loc[18:23:5]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
18,0x161c,CUS_0x2dbc,March,Langep,34,486-85-3974,Engineer,143162.64,12187.22,1,5,8,1,8,7,11.1,3.0,Good,1303.01,26.5198153915,17.9166666667,No,246.9923194537,529.0383268497,High_spent_Small_value_payments,715.741367403555,Good,0,0,0,0,1,0,1,0,0,0
23,0x1621,CUS_0x2dbc,August,Langep,34,486-85-3974,Engineer,143162.64,12187.22,1,5,8,1,8,6,7.1,3.0,Good,1303.01,38.3747529478,18.3333333333,No,246.9923194537,263.1741631616,High_spent_Small_value_payments,895.494583180492,Standard,0,0,0,0,1,0,1,0,0,0


### Fix "Num_of_Delayed_Payment" Column

In [84]:
print(f"Num_of_Delayed_Payment Mean: {cs_df.dropna(subset=['Num_of_Delayed_Payment'])['Num_of_Delayed_Payment'].str.rstrip('_').astype(int).mean()}")
delayed_payment_list = cs_df.dropna(subset=["Num_of_Delayed_Payment"])["Num_of_Delayed_Payment"].str.rstrip("_").astype(int).unique().tolist()
print(f"Highest values: {sorted(delayed_payment_list,reverse=True)[:5]}")
print(f"Lowest values: {sorted(delayed_payment_list)[:5]}")

Num_of_Delayed_Payment Mean: 30.923342437471774
Highest values: [4397, 4388, 4384, 4360, 4344]
Lowest values: [-3, -2, -1, 0, 1]


In [85]:
original_mean = cs_df.dropna(subset=["Num_of_Delayed_Payment"])["Num_of_Delayed_Payment"].str.rstrip("_").astype(int).mean()

cs_df["Num_of_Delayed_Payment"] = cs_df["Num_of_Delayed_Payment"].fillna("9999")

cs_df["Num_of_Delayed_Payment"] = cs_df["Num_of_Delayed_Payment"].apply(lambda x: x.rstrip("_")).astype(int)

outliers = (cs_df["Num_of_Delayed_Payment"] > original_mean) | (cs_df["Num_of_Delayed_Payment"] < 0)

cs_df.loc[outliers,"Num_of_Delayed_Payment"] = np.nan

In [86]:
cs_df["Num_of_Delayed_Payment"] = cs_df.groupby("Customer_ID")["Num_of_Delayed_Payment"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

In [87]:
cs_df[cs_df["Num_of_Delayed_Payment"].isna()]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data


### Fix Total_EMI_per_month Column

Terdapat beberapa nilai pada kolom Total_EMI dengan nilai yang sangat besar. Anomali disini ditandakan dengan nilai yang bahkan melebihi 2x lipat gaji bulanan. Dengan demikian, nilai-nilai anomali tersebut akan disubstitusikan dengan mode customer_ID tersebut

In [90]:
cs_df.groupby("Customer_ID")["Total_EMI_per_month"].nunique().sort_values()

Customer_ID
CUS_0x1000    1
CUS_0x82bd    1
CUS_0x82be    1
CUS_0x82bf    1
CUS_0x82c1    1
             ..
CUS_0x73cf    4
CUS_0x6db3    5
CUS_0xc2da    5
CUS_0x5262    5
CUS_0x9423    5
Name: Total_EMI_per_month, Length: 12500, dtype: int64

In [91]:
cs_df[cs_df["Customer_ID"] == "CUS_0xad12"][["Customer_ID","Total_EMI_per_month","Monthly_Inhand_Salary"]]

,Customer_ID,Total_EMI_per_month,Monthly_Inhand_Salary
79560,CUS_0xad12,186.6622120496,6807.14
79561,CUS_0xad12,186.6622120496,6807.14
79562,CUS_0xad12,54186.0000000000,6807.14
79563,CUS_0xad12,186.6622120496,6807.14
79564,CUS_0xad12,186.6622120496,6807.14
79565,CUS_0xad12,2475.0000000000,6807.14
79566,CUS_0xad12,60534.0000000000,6807.14
79567,CUS_0xad12,186.6622120496,6807.14


In [92]:
is_anomaly = cs_df["Total_EMI_per_month"] > cs_df["Monthly_Inhand_Salary"] * 2

fix_values_map = cs_df[~is_anomaly].groupby('Customer_ID')['Total_EMI_per_month'].apply(lambda x: x.mode().iloc[0] if not x.empty else np.nan)

fix_series = cs_df['Customer_ID'].map(fix_values_map)

cs_df.loc[is_anomaly,'Total_EMI_per_month'] = fix_series

In [93]:
temp_df = cs_df.groupby("Customer_ID")["Total_EMI_per_month"].agg(lambda x:x.mode().iloc[0]).rename("Mode_EMI")
cs_df = cs_df.merge(temp_df,on="Customer_ID")
cs_df.loc[cs_df["Total_EMI_per_month"] != cs_df["Mode_EMI"],"Total_EMI_per_month"] = cs_df["Mode_EMI"]
cs_df = cs_df.drop("Mode_EMI",axis=1)

In [94]:
sorted(cs_df["Total_EMI_per_month"].unique().tolist(),reverse=True)

[1779.1032538262775,
 1701.9550125306778,
 1679.017067165261,
 1645.5293878658206,
 1642.8253877782968,
 1634.2132813235144,
 1614.8559675623724,
 1582.324555736199,
 1536.7259245333985,
 1486.3258082494085,
 1320.5490633616223,
 1314.4632886687368,
 1296.430177840502,
 1260.2468384258884,
 1247.0532503714094,
 1200.655566510982,
 1146.7705860814528,
 1130.7251751172157,
 1120.378690567111,
 1119.7229272670445,
 1113.7197128896746,
 1102.9770691571075,
 1079.8824036002302,
 1070.9739242511027,
 1066.9977841908787,
 1042.6742094601684,
 1006.0854925425556,
 999.7192788944552,
 993.4409317127752,
 988.4480913512176,
 978.3884127146954,
 955.411360487057,
 950.8845835478024,
 943.7603406808716,
 932.5648320192204,
 930.2493820694092,
 928.2537831419372,
 923.8348486030224,
 921.6865326242296,
 911.2201793459374,
 910.3314912425516,
 897.0628498690637,
 891.1500920700428,
 882.9308834418084,
 848.6793880967379,
 835.8296235591246,
 828.6670381046074,
 822.9967543641145,
 821.4144810343131,

In [95]:
cs_df[cs_df["Customer_ID"] == "CUS_0xa1ec"]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
35472,0xe5da,CUS_0xa1ec,January,Danielr,51,987-26-7773,Teacher,171292.56,12574.9400654722,1,5,9,3,7,1.00,1.4900000000000002,4.0,Good,393.01,24.4683183823,31.7500000000,No,1779.1032538263,1126.2186784315,Low_spent_Large_value_payments,267.1560022700387,Good,1,0,0,0,1,1,0,0,0,0
35473,0xe5db,CUS_0xa1ec,February,Danielr,51,987-26-7773,Teacher,171292.56,14070.3800000000,1,5,9,3,3,1.00,5.49,4.0,Good,393.01,28.8515366428,31.8333333333,No,1779.1032538263,856.7845825178,Low_spent_Medium_value_payments,546.5900981836436,Good,1,0,0,0,1,1,0,0,0,0
35474,0xe5dc,CUS_0xa1ec,March,Danielr,51,987-26-7773,Teacher,171292.56,14070.3800000000,1,5,9,3,3,1.00,5.49,4.0,Good,393.01,34.3383425785,31.9166666667,No,1779.1032538263,1406.3368800131,Low_spent_Medium_value_payments,NaN,Good,1,0,0,0,1,1,0,0,0,0
35475,0xe5dd,CUS_0xa1ec,April,Danielr,51,987-26-7773,Teacher,171292.56,12574.9400654722,1,5,9,3,3,1.75,5.49,4.0,Good,393.01,36.2035964435,32.0000000000,NM,1779.1032538263,315.1097628342,High_spent_Medium_value_payments,1058.2649178673223,Good,1,0,0,0,1,1,0,0,0,0
35476,0xe5de,CUS_0xa1ec,May,Danielr,51,987-26-7773,Teacher,171292.56,12574.9400654722,1,5,9,3,0,2.50,1.4900000000000002,4.0,Good,393.01,35.0836879663,32.0833333333,No,1779.1032538263,283.1870211097,High_spent_Medium_value_payments,1090.187659591761,Good,1,0,0,0,1,1,0,0,0,0
35477,0xe5df,CUS_0xa1ec,June,Danielr,51,987-26-7773,Teacher,171292.56,12574.9400654722,1,5,9,3,3,3.25,5.49,4.0,Good,393.01,45.1320452469,32.1666666667,No,1779.1032538263,398.8408536975,Low_spent_Large_value_payments,994.5338270040138,Good,1,0,0,0,1,1,0,0,0,0
35478,0xe5e0,CUS_0xa1ec,July,Danielr,51,987-26-7773,Teacher,171292.56,12574.9400654722,1,5,9,3,6,4.00,5.49,4.0,Good,393.01,31.8894065306,32.2500000000,No,1779.1032538263,327.9806260538,High_spent_Medium_value_payments,1045.3940546477133,Standard,1,0,0,0,1,1,0,0,0,0
35479,0xe5e1,CUS_0xa1ec,August,Danielr,52,987-26-7773,Teacher,171292.56,12574.9400654722,1,5,9,3,3,1.00,5.49,4.0,Good,393.01,40.1463122530,32.3333333333,No,1779.1032538263,503.4495450264,High_spent_Medium_value_payments,869.9251356751395,Good,1,0,0,0,1,1,0,0,0,0


### Fix Monthly_Balance Column

In [97]:
for i in cs_df["Monthly_Balance"].unique():
    try:
        float(i)
    except:
        print(i)

__-333333333333333333333333333__


In [98]:
is_anomaly = cs_df["Monthly_Balance"] == "__-333333333333333333333333333__"
cs_df[is_anomaly]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
5545,0x367f,CUS_0x9885,February,Paul Dayl,17,186-49-9679,Developer,41086.400,3660.8666666667,3,4,6,5,10,17.0,17.86,10.0,Standard,113.98,34.8265412222,11.5000000000,Yes,145.5443201838,201.1429928919,Low_spent_Medium_value_payments,__-333333333333333333333333333__,Standard,0,1,1,1,1,0,1,1,0,0
26177,0xaf63,CUS_0x5a90,February,Scuffhamq,45,264-53-1943,Engineer,29728.310,2551.3591666667,3,4,6,1,15,0.0,7.81,3.0,Good,917.0,38.1980760810,31.7500000000,No,34.4964451983,198.6115126952,Low_spent_Large_value_payments,__-333333333333333333333333333__,Good,0,0,1,0,0,0,0,0,0,0
29158,0xc0d8,CUS_0x288d,July,Selamk,55,970-19-8342,Lawyer,30748.930,2572.4108333333,1,6,12,2,10,3.0,4.21,9.0,Good,860.97,40.6342498764,19.4166666667,No,43.7277739868,79.0718067495,High_spent_Small_value_payments,__-333333333333333333333333333__,Standard,1,0,0,1,0,0,0,0,0,0
35570,0xe66c,CUS_0x85e9,March,Anna Driveri,45,288-62-7562,Musician,20929.295,1704.1079166667,5,3,9,3,14,5.0,1.99,4.0,Good,858.09,33.9436256742,21.5833333333,No,40.3981749607,220.3045519782,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard,1,0,1,0,0,1,0,0,0,0
38622,0xf84c,CUS_0x2b77,July,Longstretho,21,006-36-0111,Doctor,15167.620,1471.9683333333,4,5,3,1,4,4.0,10.36,7.0,Good,847.03,28.5418476624,27.5833333333,NM,9.9308090538,128.8758720768,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard,0,0,0,0,0,0,0,0,1,0
60009,0x1759f,CUS_0xc06e,February,radenp,30,046-72-5387,Journalist,92114.680,7811.2233333333,3,4,8,1,24,10.0,17.87,7.0,Standard,820.52,39.0894753716,20.8333333333,Yes,54.2196643369,469.9911829142,High_spent_Large_value_payments,__-333333333333333333333333333__,Standard,0,0,0,1,0,0,0,0,0,0
75251,0x1ceed,CUS_0x57f3,April,Kristen Haysq,29,577-85-0830,Mechanic,60904.590,5080.3825000000,3,4,12,2,22,14.0,18.37,8.0,Standard,2441.47,29.8787155646,12.9166666667,Yes,61.5251397145,394.8939566733,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard,0,1,0,1,0,0,0,0,0,0
82918,0x1fbd8,CUS_0x41bf,July,Doeringq,44,693-64-7611,Journalist,61990.520,4900.8766666667,10,10,25,5,18,23.0,19.32,9.0,Bad,3618.91,37.7185905591,13.6666666667,NM,355.1433958714,194.2834883915,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard,0,1,1,1,1,0,0,0,1,0
83255,0x1fdd1,CUS_0x2f7e,August,Sabina Zawadzkif,31,109-48-2095,Doctor,126353.040,10635.4200000000,4,7,19,2,10,11.0,9.43,3.0,Standard,859.77,28.4428667524,31.7500000000,No,167.0439102404,549.7661392945,Low_spent_Medium_value_payments,__-333333333333333333333333333__,Poor,0,1,0,0,0,0,0,0,1,0


In [99]:
cs_df.loc[is_anomaly,"Monthly_Balance"] = np.nan

In [100]:
cs_df["Monthly_Balance"] = cs_df["Monthly_Balance"].astype("Float64")
cs_df["Monthly_Balance"] = cs_df.groupby("Customer_ID")["Monthly_Balance"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

### Fix "Changed_Credit_Limit" Column 

In [102]:
is_anomaly = cs_df["Changed_Credit_Limit"] == "_"
cs_df.loc[is_anomaly,"Changed_Credit_Limit"] = np.nan

In [103]:
cs_df[cs_df["Changed_Credit_Limit"].isna()].head(3)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data
2,0x1604,CUS_0xd40,March,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,3,7.0,NaN,4.0,Good,809.98,28.6093520221,22.2500000000,No,49.5749492149,81.6995212646,Low_spent_Medium_value_payments,331.2098628538,Good,0,0,0,1,1,0,0,1,1,0
65,0x1663,CUS_0x4157,February,Charlie Zhur,23,070-19-1622,Doctor,114838.41,9843.8675000000,2,5,7,3,14,8.0,NaN,3.0,Good,1377.74,27.6263254868,21.4166666667,NM,226.8927919215,212.2356022085,High_spent_Large_value_payments,785.25835587,Good,0,0,0,0,1,1,0,1,0,0
66,0x1664,CUS_0x4157,March,Charlie Zhur,23,070-19-1622,Doctor,114838.41,9843.8675000000,2,5,7,3,11,11.0,NaN,3.0,Good,1377.74,35.1415673019,21.5000000000,NM,226.8927919215,470.3857956796,High_spent_Small_value_payments,547.1081623988,Good,0,0,0,0,1,1,0,1,0,0


In [104]:
cs_df["Changed_Credit_Limit"] = cs_df["Changed_Credit_Limit"].astype("float64")
cs_df["Changed_Credit_Limit"] = cs_df.groupby("Customer_ID")["Changed_Credit_Limit"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

In [105]:
cs_df[cs_df["Changed_Credit_Limit"].isna()]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data


### Fix Outstanding_Debt Column 

Kolom outstanding memiliki kesalahan tipe data yang diakibatkan adanya nilai yang mengandung "\_". Oleh karena itu, character "\_" akan dihapus dari nilai Outstanding_Debt

In [108]:
cs_df["Outstanding_Debt"] = cs_df["Outstanding_Debt"].str.strip("_")

In [109]:
cs_df["Outstanding_Debt"] = cs_df["Outstanding_Debt"].astype("float64")

### Split Payment_Behaviour Column

In [111]:
pat = r'^(High|Low)_spent_(Small|Medium|Large)_value_payments$'
cs_df[['Spending_Level', 'Transaction_Value']] = cs_df['Payment_Behaviour'].str.extract(pat)
cs_df = cs_df.drop(columns="Payment_Behaviour")
cs_df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Payday Loan,Mortgage Loan,Student Loan,Home Equity Loan,Auto Loan,Debt Consolidation Loan,Not Specified,Personal Loan,Credit-Builder Loan,No Data,Spending_Level,Transaction_Value
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,3,7.0,11.27,4.0,Good,809.98,26.8226196237,22.0833333333,No,49.5749492149,80.415295439,312.4940886794,Good,0,0,0,1,1,0,0,1,1,0,High,Small
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,-1,7.0,11.27,4.0,Good,809.98,31.9449600554,22.1666666667,No,49.5749492149,118.2802216224,284.6291624961,Good,0,0,0,1,1,0,0,1,1,0,Low,Large
2,0x1604,CUS_0xd40,March,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,3,7.0,8.77,4.0,Good,809.98,28.6093520221,22.2500000000,No,49.5749492149,81.6995212646,331.2098628538,Good,0,0,0,1,1,0,0,1,1,0,Low,Medium
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,5,4.0,6.27,4.0,Good,809.98,31.3778618696,22.3333333333,No,49.5749492149,199.4580743911,223.4513097274,Good,0,0,0,1,1,0,0,1,1,0,Low,Small
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333,3,4,3,4,6,4.0,11.27,4.0,Good,809.98,24.7973469088,22.4166666667,No,49.5749492149,41.4201530862,341.4892310322,Good,0,0,0,1,1,0,0,1,1,0,High,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.4158333333,4,6,7,2,23,7.0,11.50,3.0,Good,502.38,34.6635718093,31.5000000000,No,35.1040226104,60.9713325572,479.8662281657,Poor,0,0,1,0,1,0,0,0,0,0,High,Large
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.4158333333,4,6,7,2,18,7.0,11.50,3.0,Good,502.38,40.5656309639,31.5833333333,No,35.1040226104,54.1859502876,496.6516104353,Poor,0,0,1,0,1,0,0,0,0,0,High,Medium
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.4158333333,4,6,7,2,27,6.0,11.50,3.0,Good,502.38,41.2555222590,31.6666666667,No,35.1040226104,24.0284774486,516.8090832743,Poor,0,0,1,0,1,0,0,0,0,0,High,Large
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.4158333333,4,6,7,2,20,6.0,11.50,3.0,Good,502.38,33.6382079806,31.7500000000,No,35.1040226104,251.6725821972,319.1649785257,Standard,0,0,1,0,1,0,0,0,0,0,Low,Large


### Marco Economy Feature Scraping (OECD for Inflation and Consumer Confidence Index (CCI))

In [113]:
in_cci_df = pd.read_csv("dataset/macro_economy/in_cci.csv")
in_cci_df = in_cci_df.rename(columns={"OBS_VALUE":"value"})

in_cci_df = pd.DataFrame(in_cci_df.reset_index())
in_cci_df["TIME_PERIOD"] = pd.to_datetime(in_cci_df["TIME_PERIOD"], errors='coerce')
in_cci_df["MONTH"] = in_cci_df["TIME_PERIOD"].dt.month

In [114]:
import datetime as dt
in_cci_df = pd.DataFrame(in_cci_df.reset_index())

in_cci_df["TIME_PERIOD"] = pd.to_datetime(in_cci_df["TIME_PERIOD"], errors='coerce')
in_cci_df["MONTH"] = in_cci_df["TIME_PERIOD"].dt.month

inflation_df = in_cci_df[in_cci_df["MEASURE"] == "IN"]
cci_df = in_cci_df[in_cci_df["MEASURE"] == "CCICP"]

In [115]:
print("Inflation Data Range:")
print("Lowest Value:", inflation_df["value"].quantile(0))
print("Q1:", inflation_df["value"].quantile(0.25))
print("Median:", inflation_df["value"].quantile(0.5))
print("Q3:", inflation_df["value"].quantile(0.75))
print("Highest Value:", inflation_df["value"].quantile(1))

print()

print("Consumer Confidence Index Data Range:")
print("Lowest Value:", cci_df["value"].quantile(0))
print("Q1:", cci_df["value"].quantile(0.25))
print("Median:", cci_df["value"].quantile(0.5))
print("Q3:", cci_df["value"].quantile(0.75))
print("Highest Value:", cci_df["value"].quantile(1))

Inflation Data Range:
Lowest Value: -19.3
Q1: 15.975
Median: 30.2
Q3: 45.5
Highest Value: 87.4

Consumer Confidence Index Data Range:
Lowest Value: -57.9
Q1: -20.0
Median: -12.8
Q3: 0.7
Highest Value: 127.0


In [116]:
from sklearn.preprocessing import StandardScaler

cci_df['value_z'] = cci_df.groupby('REF_AREA')["value"].transform(
    lambda x: StandardScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
)

monthly_trends_cci = (
    cci_df.groupby("MONTH")["value_z"]
    .mean()
    .reset_index(name="cci_scaled")
    .sort_values("MONTH")
)

monthly_trends_cci["cci_scaled"] = monthly_trends_cci["cci_scaled"]*100
monthly_trends_cci

/var/folders/2y/62bvs2vn73d5dy_1dvy2tb_w0000gn/T/ipykernel_7165/48110519.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cci_df['value_z'] = cci_df.groupby('REF_AREA')["value"].transform(


,MONTH,cci_scaled
0,1,2.0788679092
1,2,8.2630336755
2,3,-6.5430069683
3,4,-4.6359964998
4,5,9.2702764679
5,6,8.1269169702
6,7,1.0182945896
7,8,-0.2696454719
8,9,-2.8041658260
9,10,-6.0499455946


In [117]:
cci_labels = [
    0, #Very Pessimistic
    1, #Pessimistic
    2, #Optimistic
    3, #Very Optimistic
]
cci_bins = [
    monthly_trends_cci["cci_scaled"].quantile(0)-1,
    monthly_trends_cci["cci_scaled"].quantile(0)/2,
    0,
    monthly_trends_cci["cci_scaled"].quantile(1)/2,
    monthly_trends_cci["cci_scaled"].quantile(1)+1]
monthly_trends_cci['cci_category'] = pd.cut(monthly_trends_cci["cci_scaled"], bins=cci_bins, labels=cci_labels)
monthly_trends_cci

,MONTH,cci_scaled,cci_category
0,1,2.0788679092,2
1,2,8.2630336755,3
2,3,-6.5430069683,0
3,4,-4.6359964998,1
4,5,9.2702764679,3
5,6,8.1269169702,3
6,7,1.0182945896,2
7,8,-0.2696454719,1
8,9,-2.8041658260,1
9,10,-6.0499455946,0


In [118]:
from sklearn.preprocessing import StandardScaler



inflation_df['value_z'] = inflation_df.groupby('REF_AREA')["value"].transform(
    lambda x: StandardScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
)

monthly_trends_inflation = (
    inflation_df.groupby("MONTH")["value_z"]
    .mean()
    .reset_index(name="inflation_scaled")
    .sort_values("MONTH")
)

monthly_trends_inflation["inflation_scaled"] = monthly_trends_inflation["inflation_scaled"]*100
monthly_trends_inflation

/var/folders/2y/62bvs2vn73d5dy_1dvy2tb_w0000gn/T/ipykernel_7165/3569290354.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inflation_df['value_z'] = inflation_df.groupby('REF_AREA')["value"].transform(


,MONTH,inflation_scaled
0,1,-4.1045772927
1,2,-8.5554564206
2,3,22.9549568310
3,4,11.3313435442
4,5,7.4503395278
5,6,-1.7477487357
6,7,-10.0991034540
7,8,-15.1695462623
8,9,-7.5650509321
9,10,8.7040780157


In [119]:
inflation_labels = [
    0, #High Inflation
    1, #Inflation
    2, #Deflation
    3, #High Deflation
]
inflation_bins = [
    monthly_trends_inflation["inflation_scaled"].min()-1,
    monthly_trends_inflation["inflation_scaled"].min()/2,
    0,
    monthly_trends_inflation["inflation_scaled"].max()/2,
    monthly_trends_inflation["inflation_scaled"].max()+1] 
monthly_trends_inflation['in_category'] = pd.cut(monthly_trends_inflation['inflation_scaled'], bins=inflation_bins, labels=inflation_labels)
monthly_trends_inflation

,MONTH,inflation_scaled,in_category
0,1,-4.1045772927,1
1,2,-8.5554564206,0
2,3,22.9549568310,3
3,4,11.3313435442,2
4,5,7.4503395278,2
5,6,-1.7477487357,1
6,7,-10.0991034540,0
7,8,-15.1695462623,0
8,9,-7.5650509321,1
9,10,8.7040780157,2


### Unemployement Rate (OECD)

In [121]:
unemployement_df = pd.read_csv("dataset/macro_economy/unemployement.csv")
unemployement_df = unemployement_df.rename(columns={"OBS_VALUE":"value"}) 


unemployement_df = pd.DataFrame(unemployement_df.reset_index())
unemployement_df["TIME_PERIOD"] = pd.to_datetime(unemployement_df["TIME_PERIOD"], errors='coerce')
unemployement_df["MONTH"] = unemployement_df["TIME_PERIOD"].dt.month

In [122]:
unemployement_df['value_z'] = unemployement_df.groupby('REF_AREA')["value"].transform(
    lambda x: StandardScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
)

monthly_trends_unemployement = (
    unemployement_df.groupby("MONTH")["value_z"]
    .mean()
    .reset_index(name="unemployement_scaled")
    .sort_values("MONTH")
)
monthly_trends_unemployement["unemployement_scaled"] = monthly_trends_unemployement["unemployement_scaled"]*100

In [123]:
unemployement_labels = [
    0, #High Employement
    1, #Low Employement
    2, #Low Unemployement
    3, #High Unemployement
]
unemployement_bins = [
    monthly_trends_unemployement["unemployement_scaled"].min()-1,
    monthly_trends_unemployement["unemployement_scaled"].min()/2,
    0,
    monthly_trends_unemployement["unemployement_scaled"].max()/2,
    monthly_trends_unemployement["unemployement_scaled"].max()+1] 
monthly_trends_unemployement['unem_category'] = pd.cut(monthly_trends_unemployement['unemployement_scaled'], bins=unemployement_bins, labels=unemployement_labels)
monthly_trends_unemployement

,MONTH,unemployement_scaled,unem_category
0,1,-1.6371870055,1
1,2,-8.0846596051,1
2,3,-11.5504781966,0
3,4,-14.9087519848,0
4,5,-19.2363357252,0
5,6,17.2462438665,3
6,7,16.6210641110,3
7,8,11.5037395072,3
8,9,7.8593962266,2
9,10,4.3071923333,2


### Adjusting Month in Macro Economy Trends

In [125]:
month_mapping = {
    1:"December",
    2:"January",
    3:"February",
    4:"March",
    5:"April",
    6:"May",
    7:"June",
    8:"July",
    9:"August",
    10:"September",
    11:"October",
    12:"November"
}

In [126]:
monthly_trends_unemployement["MONTH"] = monthly_trends_unemployement["MONTH"].map(month_mapping)
monthly_trends_inflation["MONTH"] = monthly_trends_inflation["MONTH"].map(month_mapping)
monthly_trends_cci["MONTH"] = monthly_trends_cci["MONTH"].map(month_mapping)
 
macro_econ_data = [monthly_trends_unemployement,monthly_trends_inflation,monthly_trends_cci]

In [127]:
monthly_trends_unemployement

,MONTH,unemployement_scaled,unem_category
0,December,-1.6371870055,1
1,January,-8.0846596051,1
2,February,-11.5504781966,0
3,March,-14.9087519848,0
4,April,-19.2363357252,0
5,May,17.2462438665,3
6,June,16.6210641110,3
7,July,11.5037395072,3
8,August,7.8593962266,2
9,September,4.3071923333,2


### Merge Data

In [129]:
final = cs_df.copy()
for data in macro_econ_data:
    category = data.columns[-1]
    final = final.merge(
        data[["MONTH",category]],
        left_on = "Month",
        right_on = "MONTH",
        how="left"
    )
    # print(final.head())
    final = final.drop("MONTH",axis=1)

In [130]:
for i in final.columns:
    print(i, final[i].dtype)

ID object
Customer_ID object
Month object
Name object
Age int64
SSN object
Occupation object
Annual_Income float64
Monthly_Inhand_Salary float64
Num_Bank_Accounts int64
Num_Credit_Card int64
Interest_Rate int64
Num_of_Loan int64
Delay_from_due_date int64
Num_of_Delayed_Payment float64
Changed_Credit_Limit float64
Num_Credit_Inquiries float64
Credit_Mix object
Outstanding_Debt float64
Credit_Utilization_Ratio float64
Credit_History_Age float64
Payment_of_Min_Amount object
Total_EMI_per_month float64
Amount_invested_monthly Float64
Monthly_Balance Float64
Credit_Score object
Payday Loan int64
Mortgage Loan int64
Student Loan int64
Home Equity Loan int64
Auto Loan int64
Debt Consolidation Loan int64
Not Specified int64
Personal Loan int64
Credit-Builder Loan int64
No Data int64
Spending_Level object
Transaction_Value object
unem_category category
in_category category
cci_category category


### Re-adjust Credit Score Column Position

In [132]:
final["Credit_Score_temp"] = final["Credit_Score"]
final = final.drop(columns=["Credit_Score"])
final = final.rename(columns={"Credit_Score_temp":"Credit_Score"})

### Save as CSV

In [134]:
final.to_csv("dataset/final_dataset.csv")